In [3]:
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)

In [4]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 
                    48.9, 33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [5]:
def model(t_u, w, b):
    return w * t_u + b

In [6]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [7]:
params = torch.tensor([1.0 , 0.0], requires_grad=True)
print(params.shape)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr=learning_rate)

In [8]:
t_p = model(t_u, params[0], params[1])
loss = loss_fn(t_p, t_c)
loss.backward()
optimizer.step()
params

In [9]:
def train(n_epochs, optimizer, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, params[0], params[1])
        loss = loss_fn(t_p, t_c)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))
    return params

In [10]:
params = torch.tensor([1.0 , 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)
train(n_epochs=5000, optimizer=optimizer, t_u=t_un, t_c=t_c)

In [18]:
# 分割数据集
print(t_u.shape)
n_samples = t_u.shape[0]
n_val = int(n_samples * 0.2)
print(n_val)
shuffled_idx = torch.randperm(n_samples)
print(shuffled_idx)
train_idx = shuffled_idx[:-n_val]
val_idx = shuffled_idx[-n_val:]
print(train_idx, val_idx)

In [19]:
train_t_u = t_u[train_idx]
train_t_c = t_c[train_idx]

val_t_u = t_u[val_idx]
val_t_c = t_c[val_idx]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [20]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, params[0], params[1])
        loss = loss_fn(train_t_p, train_t_c)
        
        val_t_p = model(val_t_u, params[0], params[1])
        loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 500 == 0 or epoch <= 3:
            print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))
            
    return params


In [23]:
params = torch.tensor([1.0 , 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    params=params,
    train_t_u=train_t_un,
    val_t_u=val_t_un,
    train_t_c=train_t_c,
    val_t_c=val_t_c,
)